In [1]:
import keras
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, SpatialDropout1D
from keras.callbacks import ModelCheckpoint
import os
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import re
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import itertools
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
import matplotlib.pyplot as plt
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split 

C:\Users\Tanvir\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
df = pd.read_csv("dataprep.csv")  

In [3]:
df['Label'] = df.Label.map({'H':0, 'N':1})

In [4]:
df.head()

Label                                          Statement
0      0  @Grumpy_P_Sloth @deanesmay feminists argue for...
1      0  1948Army of England helped the Jews to occupy ...
2      0  Dutch Leader Says Europe to collapse In 6 Week...
3      0  RT @__DeLay: The next day the Romans and the J...
4      0  RT @Bakersman_Joe: When Hitler Invited The Jew...

In [5]:
from nltk.tokenize import word_tokenize
from nltk import FreqDist
from nltk.stem import SnowballStemmer,WordNetLemmatizer
stemmer=SnowballStemmer('english')
lemma=WordNetLemmatizer()
from string import punctuation
import re

In [6]:
def clean_review(review_col):
    review_corpus=[]
    for i in range(0,len(review_col)):
        review=str(review_col[i])
        review = re.sub(r'^@', '', review, flags=re.MULTILINE)
        review=re.sub(r'[^a-zA-Z]',' ',review)
        review = re.sub(r'^https?:\/\/.*[\r\n]*', '', review, flags=re.MULTILINE)
        review = re.sub(r'RT', '', review, flags=re.MULTILINE)
        review=[stemmer.stem(w) for w in word_tokenize(str(review).lower())]
        review=[lemma.lemmatize(w) for w in word_tokenize(str(review).lower())]
        review=' '.join(review)
        review_corpus.append(review)
    return review_corpus

In [7]:
df['clean_review']=clean_review(df.Statement.values)
df.head()

Label                                          Statement  \
0      0  @Grumpy_P_Sloth @deanesmay feminists argue for...   
1      0  1948Army of England helped the Jews to occupy ...   
2      0  Dutch Leader Says Europe to collapse In 6 Week...   
3      0  RT @__DeLay: The next day the Romans and the J...   
4      0  RT @Bakersman_Joe: When Hitler Invited The Jew...   

                                        clean_review  
0  [ 'grumpi ' , 'p ' , 'sloth ' , 'deanesmay ' ,...  
1  [ 'armi ' , 'of ' , 'england ' , 'help ' , 'th...  
2  [ 'dutch ' , 'leader ' , 'say ' , 'europ ' , '...  
3  [ 'delay ' , 'the ' , 'next ' , 'day ' , 'the ...  
4  [ 'bakersman ' , 'joe ' , 'when ' , 'hitler ' ...

In [8]:
def counter(review_col):
    review_corpus=[]
    for i in range(0,len(review_col)):
        num_words = 0
        review=str(review_col[i])
        words = review.split()
        for j in words:
            num_words += 1
        review_corpus.append(num_words)
    return review_corpus

In [9]:
df['counter']=counter(df.clean_review.values)
df.head()

Label                                          Statement  \
0      0  @Grumpy_P_Sloth @deanesmay feminists argue for...   
1      0  1948Army of England helped the Jews to occupy ...   
2      0  Dutch Leader Says Europe to collapse In 6 Week...   
3      0  RT @__DeLay: The next day the Romans and the J...   
4      0  RT @Bakersman_Joe: When Hitler Invited The Jew...   

                                        clean_review  counter  
0  [ 'grumpi ' , 'p ' , 'sloth ' , 'deanesmay ' ,...       73  
1  [ 'armi ' , 'of ' , 'england ' , 'help ' , 'th...       70  
2  [ 'dutch ' , 'leader ' , 'say ' , 'europ ' , '...       64  
3  [ 'delay ' , 'the ' , 'next ' , 'day ' , 'the ...       82  
4  [ 'bakersman ' , 'joe ' , 'when ' , 'hitler ' ...       76

In [10]:
df.drop(['Statement'],axis=1,inplace=True)
df.head()

Label                                       clean_review  counter
0      0  [ 'grumpi ' , 'p ' , 'sloth ' , 'deanesmay ' ,...       73
1      0  [ 'armi ' , 'of ' , 'england ' , 'help ' , 'th...       70
2      0  [ 'dutch ' , 'leader ' , 'say ' , 'europ ' , '...       64
3      0  [ 'delay ' , 'the ' , 'next ' , 'day ' , 'the ...       82
4      0  [ 'bakersman ' , 'joe ' , 'when ' , 'hitler ' ...       76

In [11]:
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(df['clean_review'].values)
X = tokenizer.texts_to_sequences(df['clean_review'].values)
X = pad_sequences(X)

In [22]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(Dropout(0.5))
model.add(LSTM(128,dropout=0.4, recurrent_dropout=0.4,return_sequences=True))
model.add(LSTM(64,dropout=0.5, recurrent_dropout=0.5,return_sequences=False))
model.add(Dense(2,activation='sigmoid',kernel_initializer='glorot_normal'))
model.compile(loss = 'kullback_leibler_divergence', optimizer='Nadam',metrics = ['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 81, 128)           256000    
_________________________________________________________________
dropout_2 (Dropout)          (None, 81, 128)           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 81, 128)           131584    
_________________________________________________________________
lstm_4 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 130       
Total params: 437,122
Trainable params: 437,122
Non-trainable params: 0
_________________________________________________________________
None


In [23]:
from sklearn.utils import shuffle
Y = pd.get_dummies(df['Label']).values
X,Y=shuffle(X,Y,random_state=20)
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.25, random_state = 2)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(15270, 81) (15270, 2)
(5090, 81) (5090, 2)


In [24]:
X_val = X_train[:5090]
Y_val = Y_train[:5090]

In [25]:
partial_X_train = X_train[5090:]
partial_y_train = Y_train[5090:]

In [26]:
print(X_val)

[[   0    0    0 ...    0   93    1]
 [   0    0    0 ...    1  924    1]
 [   0    0    0 ...    1 1468    1]
 ...
 [   0    0    0 ...    1  103    1]
 [   0    0    0 ...    1  845    1]
 [   0    0    0 ...   28    1    1]]


In [27]:
batch_size = 64
history = model.fit(partial_X_train, 
                    partial_y_train, 
                    epochs = 15, 
                    batch_size=batch_size, 
                    validation_data=(X_val, Y_val))

Train on 10180 samples, validate on 5090 samples
Epoch 1/15
10180/10180 [==============================] - 185s 18ms/step - loss: 0.0186 - acc: 0.7397 - val_loss: 1.8069e-04 - val_acc: 0.7424
Epoch 2/15
10180/10180 [==============================] - 160s 16ms/step - loss: 1.2135e-04 - acc: 0.7408 - val_loss: 7.2183e-05 - val_acc: 0.7424
Epoch 3/15
10180/10180 [==============================] - 163s 16ms/step - loss: 5.6341e-05 - acc: 0.7407 - val_loss: 3.9548e-05 - val_acc: 0.7424
Epoch 4/15
10180/10180 [==============================] - 173s 17ms/step - loss: 3.3263e-05 - acc: 0.7409 - val_loss: 2.5386e-05 - val_acc: 0.7424
Epoch 5/15
10180/10180 [==============================] - 161s 16ms/step - loss: 2.2101e-05 - acc: 0.7408 - val_loss: 1.7589e-05 - val_acc: 0.7424
Epoch 6/15
10180/10180 [==============================] - 155s 15ms/step - loss: 1.5627e-05 - acc: 0.7408 - val_loss: 1.2709e-05 - val_acc: 0.7424
Epoch 7/15
10180/10180 [==============================] - 156s 15ms/step 

In [29]:
loss, accuracy = model.evaluate(X_train,Y_train , verbose=False)
print("Training Accuracy: {:.2f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, Y_test, verbose=False)
print("Testing Accuracy:  {:.2f}".format(accuracy))

Training Accuracy: 0.74
Testing Accuracy:  0.74


In [ ]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20,random_state=1)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
vect = CountVectorizer()

In [ ]:
vect.fit(X_train)
X_train_dtm = vect.transform(X_train)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB(alpha=2.0)

In [ ]:
%time nb.fit(X_train, y_train)

In [ ]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(C=4,intercept_scaling=2)

In [ ]:
%time logreg.fit(X_train, y_train)